In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import re
import shutil
import time
import zipfile

from pathlib import Path
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import secrets
import util


def transform_dates(match):
    """
    Transform alphanumeric dates to numeric ones.
    e.g., Apr -> 04
    """
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    month_to_num = {month: f'{i+1:02d}' for i, month in enumerate(months)}
    return month_to_num[match.group(0)]


def move_statements_to_staging():
    # Rename alphabetic months to numeric months
    input_regex = secrets.prospectors_input_regex1
    output_regex = transform_dates    
    download_dir = secrets.prospectors_download_dir1
    staging_dir = secrets.prospectors_staging_dir1
    account_dirs = secrets.prospectors_account_dirs
    account = 'Prospectors'
    util.move_from_download_to_staging(input_regex, output_regex, download_dir, staging_dir, 
                                       account_dirs, account, [])
    
    # Transform into {year}_{month} format
    input_regex = secrets.prospectors_input_regex2
    output_regex = secrets.prospectors_output_regex
    download_dir = secrets.prospectors_download_dir2
    staging_dir = secrets.prospectors_staging_dir2
    allow_patterns = secrets.prospectors_allow_patterns
    util.move_from_download_to_staging(input_regex, output_regex, download_dir, staging_dir, 
                                       account_dirs, account, allow_patterns)

    
def move_bills_to_staging():
    input_regex = secrets.prospectors_bills_input_regex
    output_regex = secrets.prospectors_bills_output_regex
    download_dir = secrets.prospectors_download_dir1
    staging_dir = secrets.prospectors_staging_dir2
    account_dirs = secrets.prospectors_account_dirs
    account = 'Prospectors_bills'
    util.move_from_download_to_staging(input_regex, output_regex, download_dir, staging_dir, 
                                       account_dirs, account, [])


def main():
    chrome_options = webdriver.ChromeOptions()

    prefs = {
        'download.default_directory': secrets.prospectors_download_dir1,
        'download.prompt_for_download': False,
        'download.directory_upgrade': True
    }
    chrome_options.add_experimental_option('prefs', prefs)

    driver = webdriver.Chrome(options=chrome_options)
    wait = WebDriverWait(driver, 120)

    driver.get(secrets.prospectors_url)
    
    userid_box = driver.find_element('id', 'user_email')
    userid_box.send_keys(secrets.prospectors_user)

    password_box = driver.find_element('id', 'user_password')
    password_box.send_keys(secrets.prospectors_password)
    
    login_button = driver.find_element('name', 'commit')
    login_button.click()

    verification_button = wait.until(EC.element_to_be_clickable((By.ID, 'send_verification_code')))
    verification_button.click()

    
    # Here we get otp challenge
    # Open new tab
    driver.execute_script("window.open('');")

    # Switch to the new tab (it will be the last one)
    driver.switch_to.window(driver.window_handles[-1])

    # Navigate to Google Voice
    driver.get('https://voice.google.com/u/o/messages')
    singin = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), 'Sign in')]")))
    singin.click()
    
    email = wait.until(EC.element_to_be_clickable((By.TAG_NAME, "input")))
    email.send_keys(secrets.voice_user)

    next = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), 'Next')]")))
    next.click()
    

    otp = input('Enter the OTP: ')
    
    # switch back to prospectors
    driver.switch_to.window(driver.window_handles[0])
    
    # enter otp
    verification_input = driver.find_element('id', 'verification_code')
    verification_input.send_keys(otp)
    button = driver.find_element('id', 'submit_verification_code')
    button.click()
    
    # Go to statements
    statements_nav = wait.until(EC.element_to_be_clickable((By.ID, 'statements')))
    statements_nav.click()
    
    # Click statement buttons
    wait.until(EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), 'Download Packet')]")))
    time.sleep(2)  # give a little extra time
    buttons = driver.find_elements(By.XPATH, "//a[contains(text(), 'Download Packet')]")
    for button in buttons:
        button.click()
        
    move_statements_to_staging()    
    
    #download "full packets"
    full_packet_links = [l for l in driver.find_elements(By.XPATH, "//a") if l.text == 'Download Full Packet']
    for fpl in full_packet_links:
        fpl.click()
    time.sleep(2)  # give a little extra time

    # unzip them, move out owner packets and bills
    download_dir = secrets.prospectors_download_dir1
    for f in os.listdir(download_dir):
        if f.endswith('zip'):
            zip_file_path = Path(download_dir) / f
            print(f'Unzipping {zip_file_path}...')
            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                basename = os.path.splitext(os.path.basename(f))[0]
                extract_to_folder = Path(download_dir) / basename
                zip_ref.extractall(extract_to_folder)

                # move owner packet
                owner_packet_path = extract_to_folder / 'Owner Packet.pdf'
                new_owner_packet_path = Path(download_dir) / (str(basename) + '.pdf')
                owner_packet_path.rename(new_owner_packet_path)

                # move bills
                for f2 in os.listdir(extract_to_folder):
                    if f2.startswith('bill'):
                        bill_path = extract_to_folder / f2
                        bill_path.rename(Path(download_dir) / f2)             
            
    move_statements_to_staging()    
    move_bills_to_staging()
    
    util.copy_to_destination(secrets.prospectors_account_dirs,
                             secrets.prospectors_staging_dir2,
                             secrets.destination_dir)
    time.sleep(5)
    shutil.rmtree(secrets.prospectors_download_dir1)
    shutil.rmtree(secrets.prospectors_staging_dir1)
    shutil.rmtree(secrets.prospectors_staging_dir2)
    print('Done!')
    
    
main()


Enter the OTP:  119363


Unzipping /Users/ericmelz/Desktop/prospectors_statement_downloads/Jun 01, 2023 to Jun 30, 2023.zip...
Unzipping /Users/ericmelz/Desktop/prospectors_statement_downloads/Jul 01, 2023 to Jul 31, 2023.zip...
Copying account Prospectors
/Volumes/home/_Documents/_Records/_By Year/_2023/Property/Spring Hill/Property Management/Statements/2023_09.pdf
/Volumes/home/_Documents/_Records/_By Year/_2023/Property/Spring Hill/Property Management/Statements/2023_08.pdf
/Volumes/home/_Documents/_Records/_By Year/_2023/Property/Spring Hill/Property Management/Statements/2023_01.pdf
/Volumes/home/_Documents/_Records/_By Year/_2023/Property/Spring Hill/Property Management/Statements/2023_02.pdf
/Volumes/home/_Documents/_Records/_By Year/_2023/Property/Spring Hill/Property Management/Statements/2023_07.pdf
Copying account Prospectors_bills
/Volumes/home/_Documents/_Records/_By Year/_2023/Property/Spring Hill/Property Management/Bills/bill_1072_1072.pdf
/Volumes/home/_Documents/_Records/_By Year/_2023/Prope